<a href="https://colab.research.google.com/github/Reennon/multigec-models/blob/main/notebooks/gemma_3_12b/multigec/multigec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

from google.colab import userdata

os.environ["GIT_TOKEN"] = userdata.get('git_token')

In [2]:
!git clone https://$GIT_TOKEN@github.com/Reennon/multigec-models.git

fatal: destination path 'multigec-models' already exists and is not an empty directory.


In [3]:
%cd multigec-models

/content/multigec-models


In [4]:
!git pull

Already up to date.


In [5]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit -q
!pip install flash-attn --no-build-isolation -q
!pip install wtpsplit==2.1.1 -q
!pip install syntok==1.4.4 -q
!pip install omegaconf -q
!pip install wandb -q
!pip install --upgrade git+https://github.com/huggingface/transformers.git -q
!pip install --upgrade trl -q
!pip install pandas numpy -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
adapters 1.1.0 requires transformers~=4.47.1, but you have transformers 4.51.0.dev0 which is incompatible.
wtpsplit 2.1.1 requires huggingface-hub==0.25.2, but you have huggingface-hub 0.30.1 which is incompatible.


In [6]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [7]:
import os

from omegaconf import OmegaConf
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
from src.utils.multigec import sentences, LANG_TO_CODE, LANG_CODE_TO_TOKEN
from src.utils.gemma_utils import formatting_prompts_func
from langchain_core.prompts import PromptTemplate

from src.instruction_templates import multigec_prompts

import torch
import wandb

from transformers import BitsAndBytesConfig
from tqdm import tqdm
from trl.trainer import ConstantLengthDataset
import pandas as pd
from datasets import Dataset
from transformers.trainer_callback import EarlyStoppingCallback

from transformers import TrainingArguments
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig

tqdm.pandas()

In [8]:
parameters = OmegaConf.load("./params/gemma_3_12b.yaml")

In [9]:
track     = "minimal"
model_name = "gemma-3-12b-it"
fine_tuned_model_name = f"gemma-3-12b-it-multigec"
experiment_name = f"multigec-{track}-{model_name}"

hf_key   = userdata.get("hf_key")
secret_wandb = userdata.get("wandb_key")
in_path  = f"/gdrive/MyDrive/multigec/datasets/multigec_{track}.csv"

# Path where the output will be saved to
out_path = f"/gdrive/MyDrive/multigec/preds/multigec_test_{track}.csv"
out_model_dir = f"/gdrive/MyDrive/multigec/models/multigec/{fine_tuned_model_name}"
QUANTIZE_4BIT = True
device   = "cuda:0"

In [10]:
wandb_project_name = f'{model_name.upper()}-multigec-{track}'

wandb.login(key = secret_wandb)

/usr/local/lib/python3.11/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rkovalch (rkovalchuk) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
login(hf_key)

In [12]:
import gc

def clear_gpu_memory():
    torch.cuda.empty_cache()
    print(gc.collect())
clear_gpu_memory()

345


In [13]:
!env TORCH_USE_CUDA_DSA=1

SHELL=/bin/bash
NV_LIBCUBLAS_VERSION=12.5.3.2-1
NVIDIA_VISIBLE_DEVICES=all
COLAB_JUPYTER_TRANSPORT=ipc
NV_NVML_DEV_VERSION=12.5.82-1
NV_CUDNN_PACKAGE_NAME=libcudnn9-cuda-12
CGROUP_MEMORY_EVENTS=/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events
NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.22.3-1+cuda12.5
NV_LIBNCCL_DEV_PACKAGE_VERSION=2.22.3-1
VM_GCE_METADATA_HOST=169.254.169.253
HOSTNAME=669f8d321552
LANGUAGE=en_US
TBE_RUNTIME_ADDR=172.28.0.1:8011
COLAB_TPU_1VM=
GCE_METADATA_TIMEOUT=3
NVIDIA_REQUIRE_CUDA=cuda>=12.5 brand=unknown,driver>=470,driver<471 brand=grid,driver>=470,driver<471 brand=tesla,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=vapps,driver>=470,driver<471 brand=vpc,driver>=470,driver<471 brand=vcs,driver>=470,driver<471 brand=vws,driver>=470,driver<471 brand=cloudgaming,driver>=470,driver<471 brand=unknown,driver>=535,dr

In [14]:
checkpoint = f"google/{model_name}"
quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
      bnb_4bit_quant_storage=torch.bfloat16,
  )
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
config = AutoConfig.from_pretrained(checkpoint)
config.text_config.use_cache = False
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    config=config,
    quantization_config=quantization_config,
    torch_dtype="bfloat16",
    device_map=device,
    attn_implementation="eager",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You have set `use_cache` to `False`, but cache_implementation is set to hybrid. cache_implementation will have no effect.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
multigec_df = pd.read_csv(in_path)
train_df = multigec_df.loc[multigec_df.loc[:, "split"] == "train"]
val_df = multigec_df.loc[multigec_df.loc[:, "split"] == "val"]

In [16]:
num_added_toks = tokenizer.add_tokens(
    [v for v in LANG_CODE_TO_TOKEN.values()],
    special_tokens=True
)
model.resize_token_embeddings(len(tokenizer))

Gemma3TextScaledWordEmbedding(262156, 3840, padding_idx=0)

In [17]:
training_dataset = Dataset.from_pandas(train_df).shuffle()
val_dataset = Dataset.from_pandas(val_df).shuffle()

seq_length = 1600

cld_train_dataset = ConstantLengthDataset(
    tokenizer=tokenizer,
    dataset=training_dataset,
    seq_length=seq_length,
    eos_token_id=tokenizer.eos_token_id,
    shuffle=True,
    append_concat_token=True,
    add_special_tokens=True,
    formatting_func=formatting_prompts_func,
)
cld_val_dataset = ConstantLengthDataset(
    tokenizer=tokenizer,
    dataset=val_dataset,
    seq_length=int(seq_length/2),
    eos_token_id=tokenizer.eos_token_id,
    shuffle=True,
    append_concat_token=True,
    add_special_tokens=True,
    formatting_func=formatting_prompts_func,
)

In [18]:
parameters.training["gradient_accumulation_steps"] = 10
parameters.training["per_device_train_batch_size"] = 3
parameters.training["per_device_eval_batch_size"] = 1
# # Try these
# parameters.training["lr_scheduler_type"] = "cosine"
# parameters.training["max_grad_norm"] = 1.0
# parameters.training["output_dir"] = "results"
# parameters.training["num_train_epochs"] = 12
# parameters.training["eval_strategy"] = "steps"
# parameters.training["save_strategy"] = "steps"        # Save model after every step
# parameters.training["metric_for_best_model"] = "eval_loss"
# parameters.training["greater_is_better"] = False
# parameters.training["group_by_length"] = False # we already use CLD
# parameters.training["save_total_limit"] = 3           # Keep only the 3 most recent checkpoints
# parameters.training["load_best_model_at_end"] = True # EarlyStoppingCallback requires load_best_model_at_end = True
# parameters.training["warmup_steps"] = 50
# parameters.training["learning_rate"] = 3e-5
# parameters.training["save_steps"] = 25
# parameters.training["weight_decay"] = 0.0
# early_stopping_patience = 75

In [ ]:
run = wandb.init(
    project=wandb_project_name,
    job_type="training",
    anonymous="allow"
)

wandb.config.update(dict(parameters.training))

peft_config = LoraConfig(
    r=parameters.lora.r,
    lora_alpha=parameters.lora.lora_alpha,
    target_modules=parameters.lora.target_modules,
    bias=parameters.lora.bias,
    task_type=parameters.lora.task_type
)
training_arguments = SFTConfig(
    **parameters.training,
    packing=True,
    max_seq_length=seq_length,
    output_dir=out_model_dir,
)
trainer = SFTTrainer(
    model=model,
    train_dataset=cld_train_dataset,
    eval_dataset=cld_val_dataset,
    peft_config=peft_config,
    args=training_arguments,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=parameters.early_stopping.early_stopping_patience)],
)
trainer.model.config.use_cache = False

trainer.train()

/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py:259: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}  # noqa
/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py:259: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}  # noqa


/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_config.py:194: DeprecationWarning: `max_seq_length` is deprecated and will be remove in version 0.20.0. Use `max_length` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_config.py:194: DeprecationWarning: `max_seq_length` is deprecated and will be remove in version 0.20.0. Use `max_length` instead.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:107: UserWarning: To copy construct from a tensor, i

Step,Training Loss,Validation Loss
25,24.490400,2.373109
50,15.474400,1.686217


Step,Training Loss,Validation Loss
25,24.490400,2.373109
50,15.474400,1.686217
75,14.334900,1.565989
100,13.670200,1.525878
125,13.438900,1.503387
150,12.969300,1.481568
175,13.055200,1.482624


/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = [torch.tensor(example["input_ids"]) for example in examples]
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:109: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = [torch.tensor(example["input_ids"]) for example in examples]


In [ ]:
from google.colab import runtime
runtime.unassign()